<a href="https://colab.research.google.com/github/lamdary/compling_hse/blob/main/SMT_Malysheva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загружаем необходимые библиотеки

In [8]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

# Подготавливаем данные

Загружаем датасет

In [11]:
# извлекаем файлы из архива
with tarfile.open('de-en.tiny.tgz', 'r:gz') as tar:
  tar.extractall()

Смотрим, что распаковали

In [12]:
!ls

de-en.tiny.de  de-en.tiny.en  de-en.tiny.tgz  sample_data


Создаем 2 выборки и токенизируем тексты по фразам

In [13]:
with open('de-en.tiny.de', 'r') as f:
  german = f.read().split('\n')[:-1]

with open('de-en.tiny.en', 'r') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

Данные языка X:
 ['Wiederaufnahme der Sitzungsperiode', 'Ich bitte Sie , sich zu einer Schweigeminute zu erheben .', '( Das Parlament erhebt sich zu einer Schweigeminute . )', 'Frau Präsidentin , zur Geschäftsordnung .', 'Wenn das Haus damit einverstanden ist , werde ich dem Vorschlag von Herrn Evans folgen .', 'Frau Präsidentin , zur Geschäftsordnung .', 'Könnten Sie mir eine Auskunft zu Artikel 143 im Zusammenhang mit der Unzulässigkeit geben ?', 'Und zwar sollen derartige Strafen trotz des Grundsatzes der relativen Stabilität verhängt werden .', 'All dies entspricht den Grundsätzen , die wir stets verteidigt haben .', 'Vielen Dank , Herr Segni , das will ich gerne tun .', 'Das ist ganz im Sinne der Position , die wir als Parlament immer vertreten haben .', 'Das ist der Fall von Alexander Nikitin .', 'Nun ist es aber so , daß er wieder angeklagt werden soll , weil der Staatsanwalt in Berufung geht .', 'Dennoch , Frau Präsidentin , wurde meinem Wunsch nicht entsprochen .', 'Deshalb mö

Разделяем выборку с помощью sklearn

In [14]:
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

> Обучающая выборка:

Текст на немецком: Es muß auch geklärt werden , was zu tun ist , wenn staatliche Beihilfen für unrechtmäßig erklärt werden .
 Его перевод на английский: Another issue is that of what we are to do when state aid is declared unlawful .


Текст на немецком: Jetzt möchte ich zur Sache selbst etwas sagen .
 Его перевод на английский: I should now like to comment on the issue itself .


Текст на немецком: Um zu funktionieren , braucht der Markt Regeln .
 Его перевод на английский: In order to function , the market needs rules .


Текст на немецком: Die Abstimmung findet morgen um 12.00 Uhr statt .
 Его перевод на английский: The vote will take place tomorrow at 12 p.m.


Текст на немецком: Nun gilt es , dieses Prinzip in die Tat umzusetzen .
 Его перевод на английский: It only remains to put this principle into practice .


Текст на немецком: Zum Beispiel ist die unkontrollierte Migration nicht ausreichend berücksichtigt worden .
 Его перевод на английский: For example 

Чистим данные

In [15]:
def tokenize(sentences):
  # функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

# токенизируем каждую выборку
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

Образец токенизированного текста: [['Another', 'issue', 'is', 'that', 'of', 'what', 'we', 'are', 'to', 'do', 'when', 'state', 'aid', 'is', 'declared', 'unlawful', '.'], ['I', 'should', 'now', 'like', 'to', 'comment', 'on', 'the', 'issue', 'itself', '.'], ['In', 'order', 'to', 'function', ',', 'the', 'market', 'needs', 'rules', '.'], ['The', 'vote', 'will', 'take', 'place', 'tomorrow', 'at', '12', 'p.m.'], ['It', 'only', 'remains', 'to', 'put', 'this', 'principle', 'into', 'practice', '.'], ['For', 'example', ',', 'uncontrolled', 'migration', 'has', 'not', 'been', 'given', 'sufficient', 'consideration', '.'], ['That', 'is', 'why', 'my', 'Group', 'moves', 'that', 'this', 'item', 'be', 'taken', 'off', 'the', 'agenda', '.'], ['Mr', 'President', ',', 'Commissioner', ',', 'I', 'would', 'like', 'to', 'thank', 'Mrs', 'Schroedter', 'for', 'an', 'excellent', 'report', '.'], ['Mr', 'Hänsch', 'represented', 'you', 'on', 'this', 'occasion', '.'], ['There', 'is', ',', 'in', 'fact', ',', 'a', 'risk',

Создаем словарь уникальных словоформ

In [16]:
x_vocab = Counter(' '.join(german).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

Словарь немецких словоформ: dict_keys(['Wiederaufnahme', 'der', 'Sitzungsperiode', 'Ich', 'bitte', 'Sie', ',', 'sich', 'zu', 'einer', 'Schweigeminute', 'erheben', '.', '(', 'Das', 'Parlament', 'erhebt', ')', 'Frau', 'Präsidentin', 'zur', 'Geschäftsordnung', 'Wenn', 'das', 'Haus', 'damit', 'einverstanden', 'ist', 'werde', 'ich', 'dem', 'Vorschlag', 'von', 'Herrn', 'Evans', 'folgen', 'Könnten', 'mir', 'eine', 'Auskunft', 'Artikel', '143', 'im', 'Zusammenhang', 'mit', 'Unzulässigkeit', 'geben', '?', 'Und', 'zwar', 'sollen', 'derartige', 'Strafen', 'trotz', 'des', 'Grundsatzes', 'relativen', 'Stabilität', 'verhängt', 'werden', 'All', 'dies', 'entspricht', 'den', 'Grundsätzen', 'die', 'wir', 'stets', 'verteidigt', 'haben', 'Vielen', 'Dank', 'Herr', 'Segni', 'will', 'gerne', 'tun', 'ganz', 'Sinne', 'Position', 'als', 'immer', 'vertreten', 'Fall', 'Alexander', 'Nikitin', 'Nun', 'es', 'aber', 'so', 'daß', 'er', 'wieder', 'angeklagt', 'soll', 'weil', 'Staatsanwalt', 'in', 'Berufung', 'geht', 'D

# **Выбираем лучшие соответствия**

# Создаем статистику соответствий
IBM 1 Expectation-Maximization (t-model)

In [17]:
# вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

round(uniform, 8)

5.7e-07

In [18]:
# t-model
t = {}

for i in range(len(X_train)):
  # начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      # создаем t-table
      t[(word_x, word_y)] = uniform

# t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | with   ->   europäisches | Вероятность: 0.0
Соответствие | with   ->   Recht | Вероятность: 0.0
Соответствие | with   ->   Dienste | Вероятность: 0.0
Соответствие | with   ->   Sicherheit | Вероятность: 0.0
Соответствие | with   ->   schaffen | Вероятность: 0.0
Соответствие | enacting   ->   Wir | Вероятность: 0.0
Соответствие | enacting   ->   dürfen | Вероятность: 0.0
Соответствие | enacting   ->   uns | Вероятность: 0.0
Соответствие | enacting   ->   freilich | Вероятность: 0.0
Соответствие | enacting   ->   nicht | Вероятность: 0.0
Соответствие | enacting   ->   damit | Вероятность: 0.0
Соответствие | enacting   ->   begnügen | Вероятность: 0.0
Соответствие | enacting   ->   , | Вероятность: 0.0
Соответствие | enacting   ->   europäisches | Вероятность: 0.0
Соответствие | enacting   ->   Recht | Вероятность: 0.0
Соответствие | enacting   ->   im | Вероятность: 0.0
Соответствие | enacting   ->   Diens

In [19]:
# количество итераций обучения
epochs = 7

In [20]:
for epoch in range(epochs):
  # начинаем обучение

  # шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        # и слоты для статистической языковой модели y
        total[word_y] = 0

  # шаг 1. Expectation
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    # собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        # обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    # обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        # подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  # шаг 2. Maximization
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

Выходные данные были обрезаны до нескольких последних строк (5000).
Соответствие | with   ->   europäisches | Вероятность: 0.071
Соответствие | with   ->   Recht | Вероятность: 0.008
Соответствие | with   ->   Dienste | Вероятность: 0.006
Соответствие | with   ->   Sicherheit | Вероятность: 0.0
Соответствие | with   ->   schaffen | Вероятность: 0.071
Соответствие | enacting   ->   Wir | Вероятность: 0.0
Соответствие | enacting   ->   dürfen | Вероятность: 0.095
Соответствие | enacting   ->   uns | Вероятность: 0.0
Соответствие | enacting   ->   freilich | Вероятность: 0.095
Соответствие | enacting   ->   nicht | Вероятность: 0.0
Соответствие | enacting   ->   damit | Вероятность: 0.002
Соответствие | enacting   ->   begnügen | Вероятность: 0.095
Соответствие | enacting   ->   , | Вероятность: 0.0
Соответствие | enacting   ->   europäisches | Вероятность: 0.095
Соответствие | enacting   ->   Recht | Вероятность: 0.011
Соответствие | enacting   ->   im | Вероятность: 0.0
Соответствие | e

# **Биграммная модель**

In [21]:
# для обучения модели объединим 2 выборки
tokens = ' '.join(german).split()

# хранилище для биграмм
bigram_model = defaultdict(list)

# собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  # инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    # пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

defaultdict(<class 'list'>, {'Wiederaufnahme': ['der'], 'der': ['Sitzungsperiode', 'Unzulässigkeit', 'relativen', 'Position', 'Fall', 'Staatsanwalt', 'Arbeitsschutzausschuß', 'Fall', 'dortigen', 'großen', 'EU-Mitgliedstaaten', 'EU', 'Lage', 'PSE-Fraktion', 'Konferenz', 'Präsidenten', 'Konferenz', 'Präsidenten', 'Beschluß', 'Sozialistischen', 'PPE-DE-Fraktion', 'Tagesordnung', 'Fraktion', 'Tagesordnung', 'gegen', 'Werbetrommel', 'Abstimmung', 'Sitzung', 'Subsidiarität', 'ersten', 'Verkehrssicherheit', 'Prüfungsvorschriften', 'Kommission', 'Straße', 'Anwendung', 'technischen', 'Transportbehälter', 'Kommission', 'morgigen', 'Kommission', 'Zusätzlichkeit', 'Kommission', 'Wirtschaftsstruktur', 'Sozialwirtschaft', 'Arbeitsplätze', 'Union', 'fragwürdigen', 'Konservativen', 'entsprechenden', 'Leitlinien', 'Erarbeitung', 'Leitlinien', 'Evaluierung', 'Ergebnisse', 'Bericht', 'Städte', 'nationalen', 'regionalen', 'Mittel', 'Politik', 'Mittel', 'Mittelauszahlung', 'Mitgliedstaaten', 'Leitlinien', 

# **Проверяем результат**
Выбираем наиболее правдоподобное предложение

In [22]:
# сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]
  return ''  # Возвращаем пустую строку, если перевод не найден

for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    translation.append(translate(token))
  print("Перевод:", ' '.join(translation))

Оригинальное предложение: Wir sollten die ganze Sache nicht weiter aufbauschen .
Перевод: We of the  comment not   .
Оригинальное предложение: Frau Ahern , wir haben das zur Kenntnis genommen .
Перевод: ,  , we have . to note note .
Оригинальное предложение: Frau Präsidentin , zur Geschäftsordnung .
Перевод: , President , to order .
Оригинальное предложение: Vielen Dank , Herr Poettering .
Перевод: Thank Thank , Mr  .
Оригинальное предложение: Die Kommission kann nur die Daten erfassen und analysieren , die ihr seitens der Mitgliedstaaten übergeben werden .
Перевод: The Commission of only the   and  , the set to the States  . .
Оригинальное предложение: Ich komme nur darauf zu sprechen , weil ich noch einmal die Position des Parlaments verdeutlichen wollte .
Перевод: I  only  ,  , because I still we the keeping of    .
Оригинальное предложение: Nach dem Vorschlag der Kommission wäre ein Grenzwert von -20 ºC ausreichend .
Перевод: According to proposal the Commission would a  .   suffic

Проверяем его на грамматичность

In [25]:
from nltk.translate.bleu_score import corpus_bleu

reference = [X_test_tokens[0]]  # Или любое другое предложение

candidate = [translate(token) for token in y_test_tokens[0]]

bleu_score = corpus_bleu([reference], [candidate])

print("BLEU Score:", bleu_score)

BLEU Score: 1.0016022933125248e-231


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [26]:
reference

[['It',
  'should',
  'not',
  'be',
  'dramatised',
  'into',
  'something',
  'more',
  'than',
  'that',
  '.']]

In [27]:
candidate

['We', 'of', 'the', '', 'comment', 'not', '', '', '.']